In [8]:
# Importar bibliotecas necessárias
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import files
import zipfile
import os
# Fazer upload do arquivo
uploaded = files.upload()
# Configuração inicial: Extrair dataset Cats vs Dogs
dataset_path = 'cats_and_dogs.zip'  # Substitua pelo nome do arquivo local
extract_path = 'cats_and_dogs'

# Extrair o arquivo ZIP
with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Diretório base do dataset
base_dir = os.path.join(extract_path, 'PetImages')

# Diretório base do dataset - Corrigido para refletir a estrutura correta
base_dir = os.path.join(extract_path) # Assume que 'PetImages' está dentro de 'cats_and_dogs'


# Configurar ImageDataGenerator para carregar imagens
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.2  # 80% treino, 20% validação
)

train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'  # Dados de treino
    # O argumento 'errors' foi removido aqui
)

val_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'  # Dados de validação
    # O argumento 'errors' foi removido aqui
)

# Carregar o modelo pré-treinado MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Congelar as camadas base
base_model.trainable = False

# Adicionar camadas personalizadas
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Reduz as dimensões espaciais
x = Dropout(0.2)(x)  # Dropout para evitar overfitting
x = Dense(128, activation='relu')(x)  # Camada densa personalizada
output = Dense(1, activation='sigmoid')(x)  # Saída binária (gatos ou cachorros)

# Criar o modelo final
model = Model(inputs=base_model.input, outputs=output)

# Compilar o modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Resumo do modelo
model.summary()

def my_generator(gen):
    while True:
        try:
            data, labels = next(gen)
            yield data, labels
        except Exception as e:
            print(f"Erro ao carregar a imagem: {e}")
            pass # Ignorar a imagem problemática

# Treinar o modelo com os dados de treino e validação
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,  # Ajuste conforme necessário
)

# Avaliar o modelo nos dados de validação
loss, accuracy = model.evaluate(val_generator)
print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")

# Salvar o modelo treinado
model.save('cats_vs_dogs_transfer_learning.h5')
print("Modelo salvo como 'cats_vs_dogs_transfer_learning.h5'")

Saving cats_and_dogs.zip to cats_and_dogs (4).zip
Found 514 images belonging to 2 classes.
Found 128 images belonging to 2 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1 (Conv2D)            │ (None, 112, 112, 32)   │            864 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bn_Conv1                  │ (None, 112, 112, 32)   │            128 │ Conv1[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1_relu (ReLU)         │ (None, 112, 112, 32)   │              0 │ bn_Conv1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise   │ (None, 112, 112, 32)   │            288 │ Conv1_relu[0][0]       │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 112, 112, 32)   │            128 │ expanded_conv_depthwi… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 112, 112, 32)   │              0 │ expanded_conv_depthwi… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project     │ (None, 112, 112, 16)   │            512 │ expanded_conv_depthwi… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project_BN  │ (None, 112, 112, 16)   │             64 │ expanded_conv_project… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand (Conv2D)   │ (None, 112, 112, 96)   │          1,536 │ expanded_conv_project… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_BN         │ (None, 112, 112, 96)   │            384 │ block_1_expand[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_relu       │ (None, 112, 112, 96)   │              0 │ block_1_expand_BN[0][… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_pad               │ (None, 113, 113, 96)   │              0 │ block_1_expand_relu[0… │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise         │ (None, 56, 56, 96)     │            864 │ block_1_pad[0][0]      │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise_BN 

 Total params: 2,422,081 (9.24 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


17/17 ━━━━━━━━━━━━━━━━━━━━ 36s 2s/step - accuracy: 0.4452 - loss: 0.8715 - val_accuracy: 0.5000 - val_loss: 0.7381
Epoch 2/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - accuracy: 0.4393 - loss: 0.7565 - val_accuracy: 0.5000 - val_loss: 0.7095
Epoch 3/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.4758 - loss: 0.7186 - val_accuracy: 0.5000 - val_loss: 0.6956
Epoch 4/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 36s 2s/step - accuracy: 0.4642 - loss: 0.7035 - val_accuracy: 0.5000 - val_loss: 0.6959
Epoch 5/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 30s 2s/step - accuracy: 0.4638 - loss: 0.7015 - val_accuracy: 0.5000 - val_loss: 0.6978
Epoch 6/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 29s 2s/step - accuracy: 0.4595 - loss: 0.7060 - val_accuracy: 0.5000 - val_loss: 0.6942
Epoch 7/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 34s 2s/step - accuracy: 0.5101 - loss: 0.7009 - val_accuracy: 0.5000 - val_loss: 0.6953
Epoch 8/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 29s 2s/step - accuracy: 0.5128 - loss: 0.6960 - val_accuracy: 0.5000 - val_loss: 0.6948
Epo

Validation Loss: 0.6946
Validation Accuracy: 0.5000
Modelo salvo como 'cats_vs_dogs_transfer_learning.h5'
